

---


# Assignment 3: Portfolio Management 



1. Load the CRSP data from `https://github.com/mgt412/course_notebooks/blob/master/Datasets/CRSP_Data.xlsx?raw=true`.
2. Compute returns and excess returns (net of risk-free rate) for all stocks. Retain only rows where the absolute value of the return is lower than 10% (to eliminate outliers).
3. For each stock in the sample, estimate the Carhart 4-factor model -- only if there are at least 50 observation of that stock. Retain, in a separate `DataFrame`, the estimated coefficients (alpha and betas).
4. Retain the five stocks with highest estimated $\alpha$ (**Group H**) and the five stocks with lowest estimated $\alpha$ (**Group L**)
5. Using stocks in **Group H**, build a portfolio that has a beta of zero on all four risk factors. What are the estimated weights? What is the $\alpha$ of this portfolio? Repeat the task for **Group L**.
6. Compare the $\alpha$ on the zero-beta portfolios for **Group H** and **Group L**. What do you conclude?

In [1]:
from pandas import *
import numpy as np

url="https://github.com/mgt412/course_notebooks/blob/master/Datasets/CRSP_Data.xlsx?raw=true"
crsp = read_excel(url,index_col=0)
crsp['date'] = to_datetime(crsp['date'], format='%Y%m%d')
crsp['RET']=crsp.groupby('TICKER')['PRC'].apply(lambda x: x.pct_change(1))
crsp['RetRf']=crsp['RET']-crsp['rf'] # excess return

crsp=crsp[crsp['RET'].map(np.abs)<10]

In [2]:
import statsmodels.regression.linear_model as smf
from statsmodels.tools.tools import add_constant

list_tickers=crsp['TICKER'].drop_duplicates().tolist()

alphas=[]
betasM=[]
betasHML=[]
betasSMB=[]
betasUMD=[]
list_tickers2=[]

for tick in list_tickers:
    
    if len(crsp[crsp.TICKER==tick])<=50:
        continue
        
    list_tickers2.append(tick)
    
    y=crsp[crsp.TICKER==tick]['RetRf']
    x=add_constant(crsp[crsp.TICKER==tick][['mktrf','hml','smb','umd']])
    
    temp=smf.OLS(y,x,missing='drop').fit(cov_type='HC2')
    
    alphas.append(temp.params['const'])
    betasM.append(temp.params['mktrf'])
    betasHML.append(temp.params['hml'])
    betasSMB.append(temp.params['smb'])
    betasUMD.append(temp.params['umd'])

coefficients=DataFrame(columns=['TICKER','alpha','betaM','betaHML','betaSMB','betaUMD'])
coefficients['TICKER']=list_tickers2
coefficients['alpha']=alphas
coefficients['betaM']=betasM
coefficients['betaHML']=betasHML
coefficients['betaSMB']=betasSMB
coefficients['betaUMD']=betasUMD

In [3]:
coefficients.sort_values(by='alpha',ascending=True)

,TICKER,alpha,betaM,betaHML,betaSMB,betaUMD
1292,PMA,-0.763700,4.166038,1.130472,-1.742982,-1.267385
552,TAP,-0.532201,-0.674568,2.915062,0.386707,0.854397
975,CFBK,-0.426702,2.194818,-1.601014,-3.748678,0.336686
573,BIO,-0.386487,0.124729,1.418882,-1.769278,0.698999
482,WSO,-0.382563,-0.672005,-3.761502,3.024677,-1.849417
...,...,...,...,...,...,...
1315,CIE,0.067313,-0.182355,3.799479,6.305080,-0.871485
492,LSI,0.067668,0.272380,2.551747,2.166630,-0.772064
67,ACP,0.078402,1.023530,-0.038317,0.374206,-0.311107
893,MNST,0.079819,0.516284,-8.784422,1.883109,-6.575379


In [4]:
top_alpha=coefficients.sort_values(by='alpha',ascending=False).reset_index(drop=True).head()
bottom_alpha=coefficients.sort_values(by='alpha',ascending=True).reset_index(drop=True).head()

In [5]:
import scipy.optimize as spo

set_stocks=top_alpha.copy()

def alpha_port(set_stocks):

    betas_matrix=np.column_stack(
        (set_stocks[['betaM','betaHML','betaSMB','betaUMD']].values,np.ones(len(set_stocks))))

    alphas_matrix=set_stocks[['alpha']].values

    w_init=[1/len(set_stocks)]*len(set_stocks) # initial weights

    def equation_system(w):
        return np.dot(w,betas_matrix)-np.concatenate((np.zeros(4),np.ones(1))) # matrix system

    def alphas(w):
        return -np.dot(w,alphas_matrix)

    cons=({'type':'eq', 'fun':lambda x: equation_system(x)})

    ret=spo.minimize(alphas, w_init, constraints=cons)
    
    return ret.x, alphas(ret.x)

In [6]:
alpha_port(top_alpha)

(array([ 0.17762543, -0.0637106 ,  0.92464115, -0.02179803, -0.01675795]),
 array([-0.08182519]))

In [7]:
alpha_port(bottom_alpha)

(array([-0.0021866 ,  0.4049986 ,  0.20490249,  0.12074593,  0.27153958]),
 array([0.45185078]))